# Submission Information:

### Team Member 1:
* UNI:  yo2258
* Name:Ye Ouyang

### Team Member 2 [optional]:
* UNI:  
* Name:

# Step0 - Import Libraries, Load Data [0 points]

This is the basic step where you can load the data and create train and test sets for internal validation as per your convinience.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, StandardScaler
from sklearn.linear_model import Ridge
from sklearn import datasets
from sklearn.neighbors import NearestCentroid
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV,RFE



dataPath = "data/data.csv"
dataraw = pd.read_csv(dataPath)

# Step1 - Exploration and Preparation [10 points]

In this step, we expect you to look into the data and try to understand it before modeling. This understanding may lead to some basic data preparation steps which are common across the two model sets required.
********************************************************************************
1) Remove the "Duration" attribute which is not related with the modeling

2) Convert categorical data to numeric data 

3) Extract traning and target

4) Split training and testing data

In [13]:
#write code below, y
data = dataraw.drop('duration', 1)

data = pd.get_dummies(data)
X = data.iloc[:,:-2]
y = data.iloc[:,-1:]    
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

def featureSelection(model):
    rfe = RFE(model,n_features_to_select=1)
    rfe.fit(X_train,y_train)
    print rfe.ranking_

# Step2 - ModelSet1 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set1:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. Any classification algorithm covered in class apart from tree-based models can be tested here.

1) Use five models: NC/KNN/SVM/SGD/Logistic Regression

2) The feature engineering/validation/feature selection have been done in the previous step

In [14]:
def modelset1():    
    # 1: Nearest Centroid    
    nc = NearestCentroid().fit(X_train, y_train)
    print (nc.score(X_test,y_test))
    
    # 2: KNN  
    knns = neighbors.KNeighborsClassifier(n_neighbors=3).fit(X_train,y_train)
    print (knns.score(X_test,y_test))
    
    # 3: SVM      
    linearsvm = SVC().fit(X_train,y_train)
    print (linearsvm.score(X_test,y_test))
    
    # 4: SGD      
    clf = SGDClassifier(loss="hinge").fit(X_train,y_train)
    print (clf.score(X_test,y_test))
    
    # 5: Logistic Regression  
    logreg = LogisticRegression(multi_class="multinomial",solver="lbfgs").fit(X_train,y_train)
    print (logreg.score(X_test,y_test))

# Step3 - ModelSet2 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set2:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. We encourage you to try decition tree, random forest and gradient boosted tree methods here and pick the one which you think works best.


1) Use five models: Decision Tree/Random Forest/Gradient boosted Tree

2) The feature engineering/validation/feature selection have been done in the previous step

In [15]:
def modelset2():    
    # 1: Decision Tree
    dt = DecisionTreeClassifier().fit(X_train,y_train)
    print (dt.score(X_test,y_test))
    
    # 2: Random Forest
    rf = RandomForestClassifier().fit(X_train,y_train)
    print(rf.score(X_test,y_test))
    
    # 3: Gradient boosted Tree
    gbc = GradientBoostingClassifier().fit(X_train,y_train)
    print(gbc.score(X_test,y_test))

# Step4 - Ensemble [20 points + 10 Bonus points]

In this step, we expect you to use the models created before and create new predictions. You should definitely try poor man's stacking but we encourage you to think of different ensemble techniques as well. We will judge your creativity and improvement in model performance using ensemble models and you can potentially earn 10 bonus points here.

1) From Modelset1: KNeighborsClassifier and LogisticRegression have good acurracy

2) From Modelset2: Decision Tree and Random Forest have good accuracy

3) Use VotingClassifier to ensemble the best results generated from the above two steps
    - LogisticRegression
    - KNeighborsClassifier
    - Decision Tree
    - Random Forest

4) ROC figure is draw here to show the performance of classification

In [16]:
def ensemble():
    # Ensemble Clustering
    voting = VotingClassifier([('logreg',LogisticRegression(C=100)),
                               ('knn',KNeighborsClassifier()),
                               ('Dt',DecisionTreeClassifier()),
                               ('RF',RandomForestClassifier())]
                              ,voting='soft')
    model = voting.fit(X_train,y_train)
    predictions = model.predict(X_test)
    print model.score(X_test,y_test) 
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, predictions)
    roc_auc = auc(false_positive_rate, true_positive_rate)

    plt.title('ROC for Ensemble Classification')
    plt.plot(false_positive_rate, true_positive_rate, 'b',label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    return voting.score(X_test,y_test)   



In [25]:
def test_accuracy():
    assert(ensemble()>0.89)

